In [ ]:
import numpy as np
import pandas as pd
import multiprocessing
from time import time
from matplotlib import pyplot as plt

In [ ]:
name = "data/05250115"

print('Reading', name+'_MCP.csv', '...')
%time MCP_original_dataframe = pd.read_csv(name+'_MCP.csv')
print('done.')
print()
print('Reading', name+'_CsI.csv', '...')
%time CsI_original_dataframe = pd.read_csv(name+'_CsI.csv')
print('done.')
print()
print('Reading', name+'_MWPC.csv', '...')
%time MWPC_original_dataframe = pd.read_csv(name+'_MWPC.csv')
print('done.')

# (nondimensionalized) raw data of each detector in ndarray form.
MCP_raw_data  = MCP_original_dataframe.values
CsI_raw_data  = CsI_original_dataframe.values
MWPC_raw_data = MWPC_original_dataframe.values

# some constants.
c     = 299.79                # c         (mm/ns)
mc2   = 0.511                 # mc^2      (MeV)
B     = 0.1                   # B_MagSpec (T)
mc_eB = mc2 / (B * c) * 1e3   # mc_eB     (mm)
m_eB  = mc_eB / c             # m_eB      (ns)

# nondimensionalize t(ns).
MCP_raw_data.T[1]    = MCP_raw_data.T[1]    / m_eB
# nondimensionalize x,y(mm).
MCP_raw_data.T[2:4]  = MCP_raw_data.T[2:4]  / mc_eB

# nondimensionalize t(ns).
CsI_raw_data.T[1]    = CsI_raw_data.T[1]    / m_eB
# nondimensionalize E(MeV).
CsI_raw_data.T[2]    = CsI_raw_data.T[2]    / mc2

# nondimensionalize t(ns).
MWPC_raw_data.T[1]   = MWPC_raw_data.T[1]   / m_eB
# nondimensionalize x,y,z(mm).
MWPC_raw_data.T[2:5] = MWPC_raw_data.T[2:5] / mc_eB

In [ ]:
# filtering event that hit 3~5 MWPCs.
# ToDo: also select hit CsI.
MWPC_hit_count = MWPC_original_dataframe.SN.value_counts()
filtered_sn = MWPC_hit_count[(3 <= MWPC_hit_count) & (MWPC_hit_count <= 5)].index.sort_values()
print('Raw data consists of', MCP_original_dataframe.shape[0], 'events.')
print('After filtering,', filtered_sn.shape[0], 'events will be retained.')

# return filtered raw data in ndarray form.
# Note:
# The first subscript of this ndarray points to the event(0~n),
# The second subscript points to SN(0) and the data of the three detectors MCP, CsI, and MWPC in this event(1~3),
# The third subscript points to one of the records of the detector in this event 
# (Here, there should be only one record for MCP and CsI, while MWPC can have several records).
# The fourth subscript points to the specific value of this record, which corresponds to each column in the original data. 
def ConstructRawDataArray(filtered_sn, MCP_raw_data, CsI_raw_data, MWPC_raw_data):
    effective_event_num = filtered_sn.shape[0]
    # the raw data array.
    raw_data = np.ndarray((effective_event_num, 4), dtype=object)
    # fill SN.
    raw_data.T[0] = filtered_sn
    # fill MCP, CsI, MWPC data.
    MCP_iter  = 0
    CsI_iter  = 0
    MWPC_iter = 0
    for i in range(effective_event_num):
        # fill MCP data of this event.
        while MCP_iter < MCP_raw_data.shape[0]: 
            if MCP_raw_data[MCP_iter][0] != filtered_sn[i]: MCP_iter += 1
            else: break
        if MCP_iter < MCP_raw_data.shape[0]:
            raw_data[i][1] = MCP_raw_data[MCP_iter][1:4]
        # fill CsI data of this event.
        CsI_this_event_list = list()
        while CsI_iter < CsI_raw_data.shape[0]: 
            if CsI_raw_data[CsI_iter][0] != filtered_sn[i]: CsI_iter += 1
            else: break
        while CsI_iter < CsI_raw_data.shape[0]: 
            if CsI_raw_data[CsI_iter][0] == filtered_sn[i]:
                CsI_this_event_list.append(CsI_raw_data[CsI_iter][1:3])
                CsI_iter += 1
            else: break
        raw_data[i][2] = np.array(CsI_this_event_list)
        # fill MWPC data of this event.
        MWPC_this_event_list = list()
        while MWPC_iter < MWPC_raw_data.shape[0]: 
            if MWPC_raw_data[MWPC_iter][0] != filtered_sn[i]: MWPC_iter+=1
            else: break
        while MWPC_iter < MWPC_raw_data.shape[0]: 
            if MWPC_raw_data[MWPC_iter][0] == filtered_sn[i]:
                MWPC_this_event_list.append(MWPC_raw_data[MWPC_iter][1:5])
                MWPC_iter += 1
            else: break
        raw_data[i][3] = np.array(MWPC_this_event_list)
    return raw_data

%time raw_data = ConstructRawDataArray(filtered_sn, MCP_raw_data, CsI_raw_data, MWPC_raw_data)

In [ ]:
# data should be nondimensionalized.
def FitCircle(x, y, verbose=False):
    x_mean = x.mean()
    y_mean = y.mean()
    u = x - x_mean
    v = y - y_mean

    u2 = np.square(u)
    v2 = np.square(v)
    uv = u * v
    
    sum_u2 = u2.sum()
    sum_v2 = v2.sum()
    sum_uv = uv.sum()
    sum_u2v_v3 = (v * (u2 + v2)).sum()
    sum_u3_uv2 = (u * (u2 + v2)).sum()

    det = sum_u2 * sum_v2 - sum_uv * sum_uv
    if det == 0:
        if verbose:
            print('(x_i, y_i) are on a straight line, or completely overlap. nan will be returned.')
        return (np.nan, np.nan, np.nan)
    
    uc = (sum_v2 * sum_u3_uv2 - sum_uv * sum_u2v_v3) / det
    vc = (sum_u2 * sum_u2v_v3 - sum_uv * sum_u3_uv2) / det
    R = np.sqrt((np.square(u - uc) + np.square(v - vc)).mean())

    return (uc + x_mean, vc + y_mean, R)

# data should be nondimensionalized.
def FitAxial(arc, z):
    arc_mean = arc.mean()
    z_mean = z.mean()
    
    tan_alpha = ((arc * z).mean() - arc_mean * z_mean) / (np.square(arc).mean() - arc_mean * arc_mean)
    z0 = z_mean - arc_mean * tan_alpha

    return (z0, tan_alpha)

# data should be nondimensionalized.
def FitHelix(MWPC_event, verbose=False):
    x = MWPC_event.T[1]
    y = MWPC_event.T[2]
    z = MWPC_event.T[3]

    # fit circle (xy prj.).
    xc, yc, R = FitCircle(x, y, verbose)

    # fit z.
    theta = np.zeros(z.shape[0])
    for i in range(theta.shape[0]):
        delta_x = x[i] - xc
        delta_y = y[i] - yc
        if delta_y > 0:
            if delta_x > 0:
                theta[i] = np.arctan(delta_x / delta_y)
            else:
                theta[i] = np.arctan(delta_x / delta_y) + 2 * np.pi
        else:
            theta[i] = np.arctan(delta_x / delta_y) + np.pi
    arc = R * theta

    z0, tan_alpha = FitAxial(arc, z)

    # scoring.
    x_hat = xc + R * np.sin(theta)
    y_hat = yc + R * np.cos(theta)
    z_hat = z0 + arc * tan_alpha

    error_sq = np.square(x - x_hat) + np.square(y - y_hat) + np.square(z - z_hat)
    sae = np.sqrt(error_sq).sum()
    sse = error_sq.sum()
    sst = (np.square(x - x.mean()) + np.square(y - y.mean()) + np.square(z - z.mean())).sum()
    r2 = 1 - sse / sst

    if verbose:
        # plotting
        label_fontsize = 16
        ticks_fontsize = 12
        # plot x-y
        plt.plot(x_hat, y_hat, 'r+:')
        plt.scatter(x, y)
        plt.axis('equal')
        plt.xticks(fontsize=ticks_fontsize)
        plt.yticks(fontsize=ticks_fontsize)
        plt.xlabel(r'$x\left/\frac{m_e c}{eB_{MS}}\right.$', fontsize=label_fontsize)
        plt.ylabel(r'$y\left/\frac{m_e c}{eB_{MS}}\right.$', fontsize=label_fontsize)
        plt.show()
        # plot full x y
        theta_plot = np.linspace(0, 6.28, 100)
        plt.plot(xc + R * np.sin(theta_plot), yc + R * np.cos(theta_plot), 'r:')
        plt.scatter(x, y)
        plt.scatter(xc, yc)
        plt.axis('equal')
        plt.xticks(fontsize=ticks_fontsize)
        plt.yticks(fontsize=ticks_fontsize)
        plt.xlabel(r'$x\left/\frac{m_e c}{eB_{MS}}\right.$', fontsize=label_fontsize)
        plt.ylabel(r'$y\left/\frac{m_e c}{eB_{MS}}\right.$', fontsize=label_fontsize)
        plt.show()
        # plot arc-z
        plt.plot(arc, z_hat, 'r+:')
        plt.scatter(arc, z)
        plt.axis('equal')
        plt.xticks(fontsize=ticks_fontsize)
        plt.yticks(fontsize=ticks_fontsize)
        plt.xlabel(r'$s_{arc}\left/\frac{m_e c}{eB_{MS}}\right.$', fontsize=label_fontsize)
        plt.ylabel(r'$z\left/\frac{m_e c}{eB_{MS}}\right.$', fontsize=label_fontsize)
        plt.show()
    
    return [xc, yc, R, np.arctan(tan_alpha), sae, sse, r2]

def AnalyzeMWPCData(raw_event, verbose=False):
    return FitHelix(raw_event[3], verbose)

AnalyzeMWPCData(raw_data[0], True)

In [ ]:
pool = multiprocessing.Pool()
%time result_MWPC = np.array(pool.map(AnalyzeMWPCData, raw_data))
pool.close()
pool.join()

In [ ]:
# %%cython
# import numpy as np
# cimport numpy as np

# def FindTOFAndRdca(raw_event):
#     # SN Rdca TOF TOF_E
#     RdcaAndTOF = np.ndarray(3, dtype=object)
#     # SN.
#     RdcaAndTOF[0] = raw_event[0]
    
#     RdcaAndTOF[2] = FindHelix(raw_event[3])
#     return RdcaAndTOF

In [ ]:
# pool = multiprocessing.Pool()
# %time fit_result = np.array(pool.map(FindTOFAndRdca, raw_data[0:4000]), dtype=object)
# pool.close()
# pool.join()